Load libraries

In [1]:
!pip install gensim --upgrade -q

     |████████████████████████████████| 24.1 MB 3.9 MB/s 


In [1]:
import gensim

In [2]:
gensim.__version__

'4.1.2'

In [3]:

import numpy as np
import pandas as pd
from sklearn import metrics
import tqdm
import nltk
import re
from operator import itemgetter
import time

EPOCHS = 225
TOPICS = 20
CHUNK_SIZE = 1000
WORKERS = 7
EVAL_PERIOD = 10

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Clean dataset

In [5]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

Load data

In [6]:
from sklearn.datasets import fetch_20newsgroups
news_group = fetch_20newsgroups(subset='train')

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

# Creating a dataframe from the loaded data
news_df = pd.DataFrame({'news': news_group_data, 
                        'class': news_group_target})

news_df.reset_index(drop=True, inplace= True)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(news_df, test_size=0.2)

Pre-process

In [7]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['news'].progress_apply(lambda x: preprocess_text(str(x)))
test_df['news_tokenized'] = test_df['news'].progress_apply(lambda x: preprocess_text(str(x)))

100%|██████████| 2263/2263 [00:14<00:00, 160.45it/s]


Bag of Words

In [8]:
train_documents = train_df['news_tokenized'].str.split()
dictionary = gensim.corpora.Dictionary(train_documents)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=20000)

test_documents = test_df['news_tokenized'].str.split()

train_corpus = [dictionary.doc2bow(document) for document in train_documents]
test_corpus = [dictionary.doc2bow(document) for document in test_documents]

In [10]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','clustering-algo', 'run#', 'state', 'AMI','ARI','NMI','time'])


NMF

In [11]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_nmf = gensim.models.nmf.Nmf(corpus=train_corpus, num_topics=TOPICS, id2word=dictionary, chunksize=CHUNK_SIZE, passes=EPOCHS, eval_every=EVAL_PERIOD, minimum_probability=0, 
                                   random_state=state, kappa=1)
    
    pred_labels = []
    for test_doc in test_corpus:
        pred_label = max(gensim_nmf[test_doc], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
        
    performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'clustering-algo':'NMF', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)
    

Run # 1
Run # 2


LDA

In [12]:
for run, state in zip(range(1, 2, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_lda = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus, num_topics=TOPICS, id2word=dictionary, chunksize=CHUNK_SIZE, workers=WORKERS, passes=EPOCHS, 
                                                     eval_every = EVAL_PERIOD, per_word_topics=True, random_state=state)
    
    pred_labels = []
    for test_doc in test_corpus:
        pred_label = max(gensim_lda[test_doc][0], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
        
    performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'clustering-algo':'LDA', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)
    # break

Run # 1


LDA Visualization

In [17]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=94ce1e18b01fc9008e537be55e925a319b58b5eb25bfdda946901131d5b82384
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [18]:
import pyLDAvis
#Creating Topic Distance Visualization 
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(gensim_lda, test_corpus, dictionary )

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [19]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.144503  0.048318       1        1  15.060550
8      0.139802 -0.021112       2        1  12.653534
10     0.132621  0.097258       3        1   8.483494
19    -0.070334 -0.049075       4        1   8.044714
13    -0.093880  0.071584       5        1   6.859005
14     0.021314  0.092556       6        1   6.210328
5      0.067420 -0.108299       7        1   5.514218
12     0.004068  0.074541       8        1   4.948616
4     -0.122433 -0.120252       9        1   4.726049
2      0.014704  0.002345      10        1   3.880733
0      0.102002  0.141291      11        1   3.551506
9     -0.211338  0.115204      12        1   3.051357
18    -0.220899 -0.010054      13        1   2.484483
7     -0.106123  0.161752      14        1   2.471068
1      0.105456 -0.082478      15        1   2.333967
3     -0.010596 -0.125282      16        1   2.123280
15     0.105329  0.083668      17        1   2.079639
17     0.020558 -0.025362      18        1   1.977690
11     0.018652 -0.228389      19        1   1.841182
16    -0.040828 -0.118216      20        1   1.704589, topic_info=          Term         Freq        Total Category  logprob  loglift
68        file   583.000000   583.000000  Default  30.0000  30.0000
149     articl  1505.000000  1505.000000  Default  29.0000  29.0000
34     univers  1160.000000  1160.000000  Default  28.0000  28.0000
14        host   949.000000   949.000000  Default  27.0000  27.0000
24        nntp   911.000000   911.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
83        know    25.784856  1191.347830  Topic20  -5.2307   0.2388
393        say    22.388012   428.617208  Topic20  -5.3720   1.1198
528  distribut    21.897051   572.385185  Topic20  -5.3941   0.8084
198       make    18.389711   869.321764  Topic20  -5.5687   0.2159
246       well    17.989949   704.325792  Topic20  -5.5907   0.4044

[1500 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
776        1  0.209177    abort
776        3  0.149412    abort
776       14  0.627532    abort
2547       4  0.457686  acceler
2547       6  0.015782  acceler
...      ...       ...      ...
7644       3  0.982298  zionism
5619       3  0.985041  zionist
11330     10  0.989785  zisfein
250        8  0.984037   zoolog
5114      11  0.999559     zuma

[5289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 11, 20, 14, 15, 6, 13, 5, 3, 1, 10, 19, 8, 2, 4, 16, 18, 12, 17])

LSI

In [13]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_lsi = gensim.models.LsiModel(corpus=train_corpus, num_topics=TOPICS, id2word=dictionary, chunksize=CHUNK_SIZE)
    
    pred_labels = []
    for test_doc in test_corpus:
        pred_label = max(gensim_lsi[test_doc], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
            
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
      
    performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'clustering-algo':'LSI', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)
    # break

Run # 1
Run # 2


HDP

In [14]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_hdp = gensim.models.hdpmodel.HdpModel(corpus=train_corpus, id2word=dictionary, chunksize=CHUNK_SIZE, random_state=state, kappa=1, alpha=0.01)
    pred_labels = []
    for test_doc in test_corpus:
        pred_label = max(gensim_lsi[test_doc], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    # topics = gensim_hdp.print_topics()
    stop = time.time()
      
    performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'clustering-algo':'HDP', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi,'time':(stop-start)}, ignore_index=True)

Run # 1
Run # 2


In [15]:
performance_metrics

feature-extraction clustering-algo run# state       AMI       ARI       NMI  \
0       Bag of Words             NMF    1     2  0.238223  0.041038  0.264692   
1       Bag of Words             NMF    2     4  0.278979  0.069524  0.302633   
2       Bag of Words             LDA    1     2  0.454268  0.280901  0.470963   
3       Bag of Words             LSI    1     2  0.083412  0.004474  0.103052   
4       Bag of Words             LSI    2     4  0.083412  0.004474  0.103052   
5       Bag of Words             HDP    1     2  0.083412  0.004474  0.103052   
6       Bag of Words             HDP    2     4  0.083412  0.004474  0.103052   

          time  
0   200.955108  
1   201.313899  
2  1646.696481  
3     7.678815  
4     7.454431  
5    71.597439  
6    63.191797

Save metrics

In [16]:
performance_metrics.to_csv('/content/drive/MyDrive/Big Data/Code/Metrics/evaluating_performance-bow.csv', index=False)